## Intro

This is inspired by 
Article (likas2001probability) Likas, A. Probability density estimation using artificial neural networks Computer physics communications, Elsevier, 2001, 135, 167-175

But rather than estimating the working with a network, we will instead work with its derivitive.
This will let us replace their integration with a derivative.

Note that this method only works for compact supports



they use the PDF is given by $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p) dz}$$
and in their case $h=N(x,p)$  a neural network with weight and bias parameters $p$.
Where $S$ is a compact support. (That means bounded)


But if instead we say $h=\frac{\partial N(x,p)}{\partial x}$,

then $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p)}=\dfrac{\frac{\partial N(x,p)}{\partial x}}{N(max(S),p) - N(min(S), p)}$$

The denominator is ofcourse more complex for non-1D values of S.


The loss function given is the negative log-likelihood of the set of training samples $X$
$$L(p) = -\sum_{\forall x \in X} ln(h(x,p))  + |X| ln(\int_S h(z,p) dx)$$

Which befomes:

$$L(p) = -\sum_{\forall x \in X} log(\frac{\partial N(x,p)}{\partial x})  + |X|(ln(N(max(S),p)-N(min(S),p)) dx$$

In [1]:
using StatsBase
using Distributions

In [2]:
using TensorFlow
using MLDataUtils

In [ ]:
using DensityEstimationML
using Plots

using MacroTools
macro plot(ex)
    @capture(ex, (x_, [ys__], tail__ )) 
    labels = repr.(ys)
    ys_expr = Expr(:hvect, ys...)
    labels_expr = Expr(:vect, repr.(ys)...)
    Expr(:call, :plot, x, ys_expr, tail..., Expr(:kw, :labels, labels_expr))
end



In [ ]:
immutable NeuralDensityEstimator
    sess::Session
    
    #Network nodes
    optimizer::Tensor
    t::Tensor
    pdf::Tensor
end

In [ ]:
function Distributions.pdf(est::NeuralDensityEstimator, t::Real)
    gr = est.sess.graph
    ts = reshape([t], (1,1))
    run(est.sess, est.pdf, Dict(est.t=>ts))[1]
end

In [ ]:
function StatsBase.fit!(estimator::NeuralDensityEstimator, observations;
    epochs = 20,
    batch_size = 1024)
    
    gr = estimator.sess.graph
    for ii in 1:epochs
        loss_o = run(estimator.sess, 
            [gr["true_loss"], gr["working_loss"],gr["denominator"], estimator.optimizer],
            Dict(estimator.t=>observations'))
        println("Epoch $ii: loss: $(loss_o)")
    end
    estimator
end

In [ ]:
function NeuralDensityEstimator(prob_layer_sizes, support_min, support_max)
    sess = Session(Graph())
    @tf begin
        t = placeholder(Float32, shape=[1, -1])
        zsmin = [support_min*ones(Tensor,(1,1))]
        zsmax = [support_max*ones(Tensor,(1,1))]
        zt = [t]
        
        layer_sizes= [1; prob_layer_sizes; 1]
        act_funs = Vector{Function}(length(layer_sizes)-1)
        act_funs[:] = nn.sigmoid
        act_funs[end] = exp
        for ii in 2:length(layer_sizes)
            below_size = layer_sizes[ii-1]
            above_size = layer_sizes[ii]
            
            act_fun = act_funs[ii-1]
            
            Wii = get_variable("W_$ii", [above_size, below_size], Float32)
            bii = if ii!=length(layer_sizes)
                get_variable("b_$ii", [above_size, 1], Float32)
            else
                zeros(Tensor, 1,1) #Final layer has no bias
            end
            push!(zt, act_fun(Wii*zt[end] .+ bii))
            push!(zsmin, act_fun(Wii*zsmin[end] .+ bii))
            push!(zsmax, act_fun(Wii*zsmax[end] .+ bii))
        end
        
        ysmin = zsmin[end]
        ysmax = zsmax[end]
        yt = zt[end]
        
        
        denominator = (ysmax-ysmin) #area
        numerator = identity(gradients(yt,t))
        pdf =numerator/denominator
        
        n_points = TensorFlow.shape(t)[2]
        true_loss=-reduce_sum(log(numerator)) + n_points.*log(denominator)
        
        area_loss = (1f0.-denominator)^2
        working_loss = true_loss + 0.1*area_loss
        optimizer = train.minimize(train.AdamOptimizer(), working_loss)
    end
    
    run(sess, global_variables_initializer())
    
    NeuralDensityEstimator(sess, optimizer, t, pdf)
end

In [ ]:
dataset = GenerateDatasets.magdon_ismail_and_atiya
data = dataset()
est = NeuralDensityEstimator([1024], support(dataset)...)
fit!(est, data; epochs=10_000)

In [ ]:
dataset = GenerateDatasets.magdon_ismail_and_atiya
data = dataset()
est = NeuralDensityEstimator([1024], -16,16)
fit!(est, data; epochs=10_000)

In [ ]:
plot(x->pdf(est,x), xlims= support(dataset))

In [ ]:
?TensorFlow.Ops.select

In [ ]:
plot(x->x<0? -x:log(x))

In [ ]:
log(1)